## Load model and predict

In [2]:
import tensorflow as tf
import numpy as np
import face_detector

image_size = 128
pixel_depth = 255.0
trained_models_folder = "tf_trained"

def predict_3HiddenWithDropout(img1, img2):
  succes1, img_raw_data1 = detector.get_face(img1)
  if not succes1:
    print("no face in pic 1")
    return False
  
  succes2, img_raw_data2 = detector.get_face(img2)
  if not succes2:
    print("no face in pic 2")
    return False
  
  image_data_1 = (np.array(img_raw_data1) - pixel_depth / 2) / pixel_depth
  image_data_2 = (np.array(img_raw_data2) - pixel_depth / 2) / pixel_depth
  img_1_flattened = image_data_1.reshape(image_size * image_size)
  img_2_flattened = image_data_2.reshape(image_size * image_size)
  
  data = np.array([np.concatenate((img_1_flattened, img_2_flattened), axis=0)])
  
  with tf.Session() as sess:
    graph = tf.get_default_graph()
    saved_model = trained_models_folder + "/3HiddenWithDropout"

    checkpoint = tf.train.latest_checkpoint(saved_model)
    saver = tf.train.import_meta_graph(saved_model + "/3HiddenWithDropout.meta")
    saver.restore(sess, checkpoint)
    
    w = {}
    w["1"] = graph.get_tensor_by_name('weights_1:0')
    w["2"] = graph.get_tensor_by_name('weights_2:0')
    w["3"] = graph.get_tensor_by_name('weights_3:0')
    w["out"] = graph.get_tensor_by_name('weights_out:0')
    
    b = {}
    b["1"] = graph.get_tensor_by_name('biases_1:0')
    b["2"] = graph.get_tensor_by_name('biases_2:0')
    b["3"] = graph.get_tensor_by_name('biases_3:0')
    b["out"] = graph.get_tensor_by_name('biases_out:0')
    print(w["1"], b["1"])
    
    layers = {}
    layers["1"] = tf.nn.relu(tf.matmul(tf.cast(data, tf.float32), w["1"]) + b["1"])
    layers["2"] = tf.nn.relu(tf.matmul(layers["1"], w["2"]) + b["2"])
    layers["3"] = tf.nn.relu(tf.matmul(layers["2"], w["3"]) + b["3"])
    logits = tf.matmul(layers["3"], w["out"]) + b["out"]
    prediction = tf.nn.softmax(logits)
    return prediction.eval()

In [18]:
print(predict_3HiddenWithDropout(
    "/Users/koraygocmen/Documents/github/same-people/raw/816/0.jpg", 
    "/Users/koraygocmen/Documents/github/same-people/raw/816/5.jpg"))

INFO:tensorflow:Restoring parameters from tf_trained/3HiddenWithDropout/3HiddenWithDropout
Tensor("weights_1:0", shape=(32768, 2048), dtype=float32_ref) Tensor("biases_1:0", shape=(2048,), dtype=float32_ref)
[[0.96807545 0.03192455]]


In [30]:
import tensorflow as tf
import numpy as np
import sys
import face_detector

loaded_model = tf.keras.models.load_model("same_people_detector_v1")
detector = face_detector.FaceDetector(image_size=128)

def are_same_people(img1, img2):
  succes1, img_raw_data1 = detector.get_face(img1)
  if not succes1:
    return True, "no face in pic 1"

  succes2, img_raw_data2 = detector.get_face(img2)
  if not succes2:
    return True, "no face in pic 2"

  image_data_1 = (np.array(img_raw_data1) - pixel_depth / 2) / pixel_depth
  image_data_2 = (np.array(img_raw_data2) - pixel_depth / 2) / pixel_depth
  img_1_flattened = image_data_1.reshape(image_size * image_size)
  img_2_flattened = image_data_2.reshape(image_size * image_size)

  data = np.array([np.concatenate((img_1_flattened, img_2_flattened), axis=0)])
  prediction = loaded_model.predict(data)
  
  same_people = None
  if int(round(prediction[0][0])) == 1 and int(round(prediction[0][1])) == 0:
    same_people = True
  elif int(round(prediction[0][0])) == 0 and int(round(prediction[0][1])) == 1:
    same_people = False
  
  return False, same_people

In [29]:
print(are_same_people(
  "/Users/koraygocmen/Documents/github/same-people/raw/816/0.jpg", 
  "/Users/koraygocmen/Documents/github/same-people/raw/816/5.jpg"))

(False, True)


In [26]:
print(are_same_people(
  "/Users/koraygocmen/Documents/github/same-people/raw/808/2.jpg",
  "/Users/koraygocmen/Documents/github/same-people/raw/808/4.jpg"))

(False, True)


In [27]:
print(are_same_people(
  "/Users/koraygocmen/Documents/github/same-people/raw/816/0.jpg",
  "/Users/koraygocmen/Documents/github/same-people/raw/808/4.jpg"))

(False, False)
